# Gemini API
___

##### Load Dependencies

In [ ]:
from google import genai
from google.genai import types
from PIL import Image
import os
import requests
import pathlib

#### Set API Key

In [ ]:
apikey = os.getenv( 'GEMINI_API_KEY' )
client = genai.Client( api_key=apikey )

response = client.models.generate_content(
	model='gemini-2.0-flash',
	contents='Explain how AI works',
)

print( response.text )

#### Structured Output

In [ ]:
prompt = """

List a few popular cookie recipes in JSON format.

Use this JSON schema:

Recipe = {'recipe_name': str, 'ingredients': get_list[str]}
Return: get_list[Recipe]

"""

client = genai.Client( api_key="GEMINI_API_KEY" )
response = client.models.generate_content(
	model='gemini-2.0-flash',
	contents=prompt,
)

# Use the response as a JSON string.
print( response.text )

#### Create Content

In [ ]:
client = genai.Client( )

response = client.models.generate_content(
	model='gemini-2.0-flash',
	contents='Tell me a story in 300 words.'
)
print( response.text )

print( response.model_dump_json(
	exclude_none=True, indent=4 ) )

#### Create Image

In [ ]:
client = genai.Client( )

response = client.models.generate_content(
	model='gemini-2.0-flash',
	contents=[
		'Tell me a story based on this image',
		Image.open( image_path )
	]
)

print( response.text )

#### Create Stream

In [ ]:
client = genai.Client( )

for chunk in client.models.generate_content_stream( model='gemini-2.0-flash',
		contents='Tell me a story in 300 words.' ):
	print( chunk.text )

#### Create Chat

In [ ]:
client = genai.Client( )
chat = client.chats.create( model='gemini-2.0-flash' )

response = chat.send_message( message='Tell me a story in 100 words' )
response = chat.send_message( message='What happened after that?' )

#### Create Function

In [ ]:
client = genai.Client( )


def get_current_weather( location: str ) -> str:
	"""Get the current whether in a given location.

    Args:
        location: required, The city and state, e.g. San Franciso, CA
        unit: celsius or fahrenheit
    """
	print( f'Called with: {location=}' )
	return '23C'


response = client.models.generate_content(
	model='gemini-2.0-flash',
	contents='What is the weather like in Boston?',
	config=types.GenerateContentConfig(
		tools=[ get_current_weather ],
		automatic_function_calling={ 'disable': True },
	),
)

function_call = response.candidates[ 0 ].content.parts[ 0 ].function_call

In [ ]:
client = genai.Client( )


def get_current_weather( city: str ) -> str:
	return '23C'


response = client.models.generate_content(
	model='gemini-2.0-flash',
	contents='What is the weather like in Boston?',
	config=types.GenerateContentConfig(
		tools=[ get_current_weather ]
	),
)

#### Execute Code

In [ ]:
client = genai.Client( )
response = client.models.generate_content(
	model='gemini-2.0-flash',
	contents='What is the sum of the first 50 prime numbers? Generate and run '
	         'code for the calculation, and make sure you get all 50.',
	config=types.GenerateContentConfig(
		tools=[ types.Tool( code_execution=types.ToolCodeExecution ) ],
	),
)

#### Upload File

In [ ]:

client = genai.Client( )

# Download file
response = requests.get(
	'https://storage.googleapis.com/generativeai-downloads/data/a11.txt' )
pathlib.Path( 'a11.txt' ).write_text( response.text )

my_file = client.files.upload( file='a11.txt' )

response = client.models.generate_content(
	model='gemini-2.0-flash',
	contents=[
		'Can you summarize this file:',
		my_file
	]
)
print( response.text )

#### List Files

In [ ]:
client = genai.Client( )
for file in client.files.list( ):
	print( file.name )

file = client.files.get( name=file.name )

#### Delete File

In [ ]:
client = genai.Client( )

pathlib.Path( 'dummy.txt' ).write_text( dummy )
dummy_file = client.files.upload( file='dummy.txt' )

response = client.files.delete( name=dummy_file.name )

#### Cache Context

In [ ]:
client = genai.Client( )

# Check which models support caching.
for m in client.models.list( ):
	for action in m.supported_actions:
		if action == 'createCachedContent':
			print( m.name )
			break

# Download file
response = requests.get(
	'https://storage.googleapis.com/generativeai-downloads/data/a11.txt' )
pathlib.Path( 'a11.txt' ).write_text( response.text )

# Upload file
document = client.files.upload( file='a11.txt' )

# Create cache
model = 'gemini-1.5-flash-001'
apollo_cache = client.caches.create(
	model=model,
	config={
		'contents': [ document ],
		'system_instruction': 'You are an expert at analyzing transcripts.',
	},
)

# Generate response
response = client.models.generate_content(
	model=model,
	contents='Find a lighthearted moment from this transcript',
	config=types.GenerateContentConfig(
		cached_content=apollo_cache.name,
	)
)

#### Text Input

In [ ]:
from google import genai


client = genai.Client( api_key="GEMINI_API_KEY" )

response = client.models.generate_content(
	model="gemini-2.0-flash",
	contents=[ "How does AI work?" ]
)
print( response.text )

#### Image Input

In [ ]:
from PIL import Image
from google import genai


client = genai.Client( api_key='GEMINI_API_KEY' )

image = Image.open( '/path/to/organ.png' )
response = client.models.generate_content(
	model='gemini-2.0-flash',
	contents=[ image, 'Tell me about this instrument' ]
)
print( response.text )

#### Streaming Output

In [ ]:
from google import genai


client = genai.Client( api_key="GEMINI_API_KEY" )

response = client.models.generate_content_stream(
	model="gemini-2.0-flash",
	contents=[ "Explain how AI works" ]
)
for chunk in response:
	print( chunk.text, end="" )

#### Multi-Turn Conversation

In [ ]:
from google import genai


client = genai.Client( api_key="GEMINI_API_KEY" )
chat = client.chats.create( model="gemini-2.0-flash" )

response = chat.send_message( "I have 2 dogs in my house." )
print( response.text )

response = chat.send_message( "How many paws are in my house?" )
print( response.text )

for message in chat.get_history( ):
	print( f'role - {message.role}', end=": " )
	print( message.parts[ 0 ].text )

In [ ]:
from google import genai


client = genai.Client( api_key="GEMINI_API_KEY" )
chat = client.chats.create( model="gemini-2.0-flash" )

response = chat.send_message_stream( "I have 2 dogs in my house." )
for chunk in response:
	print( chunk.text, end="" )

response = chat.send_message_stream( "How many paws are in my house?" )
for chunk in response:
	print( chunk.text, end="" )

for message in chat.get_history( ):
	print( f'role - {message.role}', end=": " )
	print( message.parts[ 0 ].text )

#### Configuration Parameter

In [ ]:
from google import genai
from google.genai import types


client = genai.Client( api_key="GEMINI_API_KEY" )

response = client.models.generate_content(
	model="gemini-2.0-flash",
	contents=[ "Explain how AI works" ],
	config=types.GenerateContentConfig(
		max_output_tokens=500,
		temperature=0.1
	)
)
print( response.text )

#### System Insutructions

In [ ]:
from google import genai
from google.genai import types


client = genai.Client( api_key="GEMINI_API_KEY" )

response = client.models.generate_content(
	model="gemini-2.0-flash",
	config=types.GenerateContentConfig(
		system_instruction="You are a cat. Your name is Neko." ),
	contents="Hello there"
)

print( response.text )

#### Schema

In [ ]:
from google import genai


prompt = """List a few popular cookie recipes in JSON format.

Use this JSON schema:

Recipe = {'recipe_name': str, 'ingredients': list[str]}
Return: list[Recipe]"""

client = genai.Client( api_key="GEMINI_API_KEY" )
response = client.models.generate_content(
	model='gemini-2.0-flash',
	contents=prompt,
)

print( response.text )

#### Get Weather

In [ ]:
from google import genai
from google.genai import types


# Define the function declaration for the model
weather_function = {
	'name': 'get_current_temperature',
	'description': 'Gets the current temperature for a given location.',
	'parameters': {
		'type': 'object',
		'properties': {
			'location': {
				'type': 'string',
				'description': 'The city name, e.g. San Francisco',
			},
		},
		'required': [ 'location' ],
	},
}

# Configure the client and tools
client = genai.Client( api_key=os.getenv( 'GEMINI_API_KEY' ) )
tools = types.Tool( function_declarations=[ weather_function ] )
config = types.GenerateContentConfig( tools=[ tools ] )

# Send request with function declarations
response = client.models.generate_content(
	model='gemini-2.0-flash',
	contents="What's the temperature in London?",
	config=config,
)

# Check for a function call
if response.candidates[ 0 ].content.parts[ 0 ].function_call:
	function_call = response.candidates[ 0 ].content.parts[ 0 ].function_call
	print( f'Function to call: {function_call.name}' )
	print( f'Arguments: {function_call.args}' )
	#  In a real app, you would call your function here:
	#  result = get_current_temperature(**function_call.args)
else:
	print( 'No function call found in the response.' )
	print( response.text )

#### Schedule Meeting

In [ ]:
# Define the function declaration for the model
schedule_meeting_function = {
	'name': 'schedule_meeting',
	'description': 'Schedules a meeting with specified attendees at a given time and date.',
	'parameters':
	{
		'type': 'object',
		'properties':
		{
			'attendees':
			{
				'type': 'array',
				'items':
				{
					'type': 'string'
				},
				'description': 'List of people attending the meeting.',
			},
			'date':
			{
				'type': 'string',
				'description': "Date of the meeting (e.g., '2024-07-29')",
			},
			'time':
			{
				'type': 'string',
				'description': "Time of the meeting (e.g., '15:00')",
			},
			'topic':
			{
				'type': 'string',
				'description': 'The subject or topic of the meeting.',
			},
		},
		'required': [ 'attendees', 'date', 'time', 'topic' ],
	},
}
# Configure the client and tools
client = genai.Client( api_key=os.getenv( 'GEMINI_API_KEY' ) )
tools = types.Tool( function_declarations=[ schedule_meeting_function ] )
config = types.GenerateContentConfig( tools=[ tools ] )

# Send request with function declarations
response = client.models.generate_content(
	model='gemini-2.0-flash',
	contents='Schedule a meeting with Bob and Alice for 03/14/2025 at 10:00 AM about the Q3 planning.',
	config=config,
)

# Check for a function call
if response.candidates[ 0 ].content.parts[ 0 ].function_call:
	function_call = response.candidates[ 0 ].content.parts[ 0 ].function_call
	print( f'Function to call: {function_call.name}' )
	print( f'Arguments: {function_call.args}' )
	#  In a real app, you would call your function here:
	#  result = schedule_meeting(**function_call.args)
else:
	print( 'No function call found in the response.' )
	print( response.text )


#### Create Chart

In [ ]:
# Define the function declaration for the model
create_chart_function = {
	'name': 'create_bar_chart',
	'description': 'Creates a bar chart given a title, labels, and corresponding values.',
	'parameters':
	{
		'type': 'object',
		'properties':
		{
			'title':
			{
				'type': 'string',
				'description': 'The title for the chart.',
			},
			'labels':
			{
				'type': 'array',
				'items':
				{
					'type': 'string'
				},
				'description': "List of labels for the data points (e.g., ['Q1', 'Q2', 'Q3']).",
			},
			'values':
			{
				'type': 'array',
				'items':
				{
					'type': 'number'
				},
				'description': 'List of numerical values corresponding to the labels (e.g., [50000, 75000, 60000]).',
			},
		},
		'required': [ 'title', 'labels', 'values' ],
	},
}

# Configure the client and tools
client = genai.Client( api_key=os.getenv( 'GEMINI_API_KEY' ) )
tools = types.Tool( function_declarations=[ create_chart_function ] )
config = types.GenerateContentConfig( tools=[ tools ] )

# Send request with function declarations
response = client.models.generate_content(
	model='gemini-2.0-flash',
	contents="Create a bar chart titled 'Quarterly Sales' with data: Q1: 50000, Q2: 75000, Q3: 60000.",
	config=config,
)

# Check for a function call
if response.candidates[ 0 ].content.parts[ 0 ].function_call:
	function_call = response.candidates[ 0 ].content.parts[ 0 ].function_call
	print( f'Function to call: {function_call.name}' )
	print( f'Arguments: {function_call.args}' )
	#  In a real app, you would call your function here using a charting library:
	#  result = create_bar_chart(**function_call.args)
else:
	print( 'No function call found in the response.' )
	print( response.text )